# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys1 import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,broken hill,-31.9500,141.4333,297.76,72,100,4.80,AU,1707094697
1,1,waitangi,-43.9535,-176.5597,289.62,79,100,5.74,NZ,1707094251
2,2,yellowknife,62.4560,-114.3525,247.44,76,20,3.09,CA,1707094516
3,3,rundu,-17.9333,19.7667,292.31,95,78,2.72,NaN,1707094638
4,4,blackmans bay,-43.0167,147.3167,294.22,62,2,0.89,AU,1707094520


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
    )

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [63]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Lat"].between(-40,40)) & (city_data_df["Humidity"]<80) & (city_data_df["Lng"]<-30)].reset_index()

# Drop any rows with null values
ideal_weather_df.dropna()

# Deleting index column that was created by reseting the index
del ideal_weather_df["index"]

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,21,adamstown,-25.0660,-130.1015,298.35,76,100,3.80,PN,1707094427
1,52,hawaiian paradise park,19.5933,-154.9731,297.52,67,100,3.60,US,1707094521
2,60,general mosconi,-23.1851,-62.3030,303.65,41,68,3.84,AR,1707094975
3,70,new albany,38.2856,-85.8241,284.71,49,75,4.63,US,1707094981
4,95,iquique,-20.2208,-70.1431,298.17,72,7,1.04,CL,1707094996
...,...,...,...,...,...,...,...,...,...,...
61,518,callao,-12.0667,-77.1500,301.33,69,0,4.63,PE,1707095282
62,519,camocim,-2.9022,-40.8411,300.71,76,20,6.92,BR,1707095283
63,526,nhamunda,-2.1861,-56.7131,299.54,74,25,1.46,BR,1707095287
64,531,santa cruz,-17.8000,-63.1667,300.85,78,12,2.57,BO,1707095291


### Step 3: Create a new DataFrame called `hotel_df`.

In [64]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,adamstown,PN,-25.0660,-130.1015,76,
1,hawaiian paradise park,US,19.5933,-154.9731,67,
2,general mosconi,AR,-23.1851,-62.3030,41,
3,new albany,US,38.2856,-85.8241,49,
4,iquique,CL,-20.2208,-70.1431,72,
...,...,...,...,...,...,...
61,callao,PE,-12.0667,-77.1500,69,
62,camocim,BR,-2.9022,-40.8411,76,
63,nhamunda,BR,-2.1861,-56.7131,74,
64,santa cruz,BO,-17.8000,-63.1667,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [65]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle: {longitude},{latitude},{radius}"
    params["bias"] = f"proximity: {longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?categories=accommodation.hotel&"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
general mosconi - nearest hotel: No hotel found
new albany - nearest hotel: No hotel found
iquique - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
trairi - nearest hotel: No hotel found
kapa'a - nearest hotel: No hotel found
itaueira - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
afaahiti - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
valparaiso - nearest hotel: No hotel found
taulaga - nearest hotel: No hotel found
carolina beach - nearest hotel: No hotel found
kill devil hills - nearest hotel: No hotel found
mahina - nearest hotel: No hotel found
mar del plata - nearest hotel: No hotel found
silver city - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
lagunillas - nearest hotel: No hotel found
san julian - ne

,City,Country,Lat,Lng,Humidity,Hotel Name
0,adamstown,PN,-25.0660,-130.1015,76,No hotel found
1,hawaiian paradise park,US,19.5933,-154.9731,67,No hotel found
2,general mosconi,AR,-23.1851,-62.3030,41,No hotel found
3,new albany,US,38.2856,-85.8241,49,No hotel found
4,iquique,CL,-20.2208,-70.1431,72,No hotel found
...,...,...,...,...,...,...
61,callao,PE,-12.0667,-77.1500,69,No hotel found
62,camocim,BR,-2.9022,-40.8411,76,No hotel found
63,nhamunda,BR,-2.1861,-56.7131,74,No hotel found
64,santa cruz,BO,-17.8000,-63.1667,78,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [66]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    scale = 0.5,
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols = ["Hotel Name","Country"]
    )

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)